In [ ]:
import h5py as h5

import numpy as np
import ufl

from mpi4py import MPI
from petsc4py.PETSc import ScalarType
from dolfinx import mesh, fem, plot, io
print(dolfin.__version__)

ModuleNotFoundError: No module named 'dolfinx'

In [ ]:
# # Things that stay the same throughout time loop in fenicsx

# filename_deformation = "data/mechanical_data/test_{}_deformation_times.xdmf".format(stem,full_stem)
# filename_T_a = "data/mechanical_data/test_{}_active_tension_times.xdmf".format(stem,full_stem)

N = len(data[-1])
L = np.max(data[-1])

mu = 1
beta = 1.25
lambda_ = beta

mesh_points = [np.array([0,0]), np.array([L,L])]

# 2d domain
domain = mesh.create_rectangle(MPI.COMM_WORLD,mesh_points,[N-1,N-1],mesh.CellType.quadrilateral)
V = fem.VectorFunctionSpace(domain, ("CG", 1))
F = fem.FunctionSpace(domain, ("CG", 1))

# boundary u = 0
def clamped_boundary_x0(x):
    return np.isclose(x[0], 0)
def clamped_boundary_x1(x):
    return np.isclose(x[0], L)
def clamped_boundary_y0(x):
    return np.isclose(x[1], 0)
def clamped_boundary_y1(x):
    return np.isclose(x[1], L)

fdim = domain.topology.dim - 1
boundary_facets_x0 = mesh.locate_entities_boundary(domain, fdim, clamped_boundary_x0)
boundary_facets_x1 = mesh.locate_entities_boundary(domain, fdim, clamped_boundary_x1)
boundary_facets_y0 = mesh.locate_entities_boundary(domain, fdim, clamped_boundary_y0)
boundary_facets_y1 = mesh.locate_entities_boundary(domain, fdim, clamped_boundary_y1)
total_boundary_facets = np.concatenate((boundary_facets_x0, boundary_facets_x1,boundary_facets_y0,boundary_facets_y1))
u_D = np.array([0,0], dtype=ScalarType)
bc = fem.dirichletbc(u_D, fem.locate_dofs_topological(V, fdim, total_boundary_facets), V)

# functions to solve the finite element system
def epsilon(u):
    return ufl.sym(ufl.grad(u)) # Equivalent to 0.5*(ufl.nabla_grad(u) + ufl.nabla_grad(u).T)
def sigma(u):
    return lambda_ * ufl.nabla_div(u) * ufl.Identity(u.geometric_dimension()) + 2*mu*epsilon(u)

dof_coordinates = V.tabulate_dof_coordinates()
coord = data[-1]

# open files
xdmf = io.XDMFFile(domain.comm, filename_deformation, "w")
xdmf.write_mesh(domain)
xdmf_T_a = io.XDMFFile(domain.comm, filename_T_a, "w")
xdmf_T_a.write_mesh(domain)

NameError: name 'stem' is not defined

In [ ]:
fenicsx

for t in range(len(T)):

    # putting T in finite function
    f = fem.Function(F)
    value_t = T[t,:,:]

    coord_flatten = coord.reshape(-1, coord.shape[-1])
    value_flatten = value_t.flatten()

    for i,dc in enumerate(dof_coordinates):
        for j,c in enumerate(coord_flatten):
            if abs(dc[0] - c[0]) < 10**(-3) and abs(dc[1] - c[1]) < 10**(-3):
                f.vector[i] = value_flatten[j]
                        
    T_finite = ufl.grad(f)

    u = ufl.TrialFunction(V)
    v = ufl.TestFunction(V)
    a = ufl.inner(sigma(u), epsilon(v)) * ufl.dx
    L = ufl.dot(T_finite, v) * ufl.dx

    problem = fem.petsc.LinearProblem(a, L, bcs=[bc], petsc_options={"ksp_type": "preonly", "pc_type": "lu"})
    uh = problem.solve()
    uh.name = "Deformation"
    f.name ="Active_Tension"

    xdmf.write_function(uh, t)
    xdmf_T_a.write_function(f, t)

    print('timestep {}/{}'.format(t,len(T)))

xdmf.close()
xdmf_T_a.close()